<a href="https://colab.research.google.com/github/hajonghyun/installPytorch_study/blob/main/5_1_short_multiclass_classificaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/ROKAF/인스톨 파이토치/torch_코드모음')
from multiclass_functions1 import * # all
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cuda


In [3]:
BATCH_SIZE = 32
LR = 1e-3
EPOCH = 5
criterion = nn.CrossEntropyLoss()
new_model_train = False
model_type = 'MLP'
dataset = 'MNIST'
save_model_path = f"/content/drive/MyDrive/ROKAF/인스톨 파이토치/results/{model_type}_{dataset}.pt"

In [4]:
transform = transforms.ToTensor()
train_DS = datasets.MNIST(root='/content/drive/MyDrive/ROKAF/인스톨 파이토치/data', train=True, download=True, transform= transform)
test_DS = datasets.MNIST(root='/content/drive/MyDrive/ROKAF/인스톨 파이토치/data', train=False, download=True, transform= transform)
train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS,
batch_size= BATCH_SIZE, shuffle=True)

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fcs = nn.Sequential(nn.Linear(28*28, 30),
                                 nn.BatchNorm1d(30),
                                 nn.ReLU(),
                                 nn.Linear(30,10))
    def forward(self,x):
        x = torch.flatten(x, start_dim=1)
        x = self.fcs(x)
        return x

In [6]:
model = globals()[model_type]().to(DEVICE)
# globals() => global 변수들이 딕셔너리 형태로. 따라서 globals()[model_type]는 model_type이 'MLP'로 설정되어있으므로 'MLP'라는 string으로 인덱싱한 것

print(model)

x_batch, _ = next(iter(train_DL))
print(x_batch.shape)

model.eval()
with torch.no_grad():
    print(model(x_batch.to(DEVICE)).shape)

MLP(
  (fcs): Sequential(
    (0): Linear(in_features=784, out_features=30, bias=True)
    (1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=30, out_features=10, bias=True)
  )
)
torch.Size([32, 1, 28, 28])
torch.Size([32, 10])
